# CF-netCDF with cfdm, cf-python and cf-plot: a demo in under an hour

----

## A ~45 minute illustration of the basic functionality of three inter-related Python libraries\* underpinned by [the CF data model](https://gmd.copernicus.org/articles/10/4619/2017/) for working with CF-netCDF.

\* these data tools are developed & maintained by the [CMS team](https://www.ncas.ac.uk/en/cms) of the National Centre for Atmospheric Science (NCAS)

A recap of the data tools and their respective scopes are:

* #### [cfdm](https://ncas-cms.github.io/cfdm/) (`cfdm` module): reference implementation of the CF data model with mostly only the functionality required to read and write datasets, and to create, modify and inspect field constructs in memory;
* #### [cf-python](https://ncas-cms.github.io/cf-python/) (`cf`): CF-compliant geoscientific data analysis library which builds upon `cfdm` to provide much higher-level functionality, for example statistical operations, collapsing, subspacing, and regridding;
* #### [cf-plot](http://ajheaps.github.io/cf-plot/) (`cfplot`): set of Python functions for making common visualisations such as contour, vector and line plots that are used often by geoscientists.

*Note*: this summary focuses on use of these tools *with netCDF (`.nc`) datasets only*, however cfdm and cf-python can recognise and map to field constructs other formats, namely CDL (`.cdl`) of netCDF and (for cf-python only) PP (`.pp`) and UM fields files (`.ff`), and cf-plot also accepts pure NumPy arrays as input.

## Learning objectives:

### ~10 minutes for each of four segments demonstrating some (but by no means all!) of the capabilities of the tools:

1. **From netCDF to field constructs and back**: read in netCDF files, create a new field construct by modification of data and metadata and then write out the new field to a new netCDF file.
2. **Basic data analysis, with plotting of results**: Plot the data before and after applying statistical collapses.
3. **Regridding domains, with plotting of results**:  plot the data before and after regridding across spherical and cartesian coordinate systems.
4. **Manipulating hierarchical groups**: create & inspect group structure for a netCDF-4 file, then flatten it out.

----

## 0. Setup

First let's setup the Notebook environment:

In [1]:
# Setup for nice outputs in this Jupyter Notebook (not required in interactive Python or a script)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Throughout this walk-through we will be working on some sample datasets, contained in `ncas_data`. Let's check what we have to work with, with a shell command:

In [2]:
# Note that in IPython ! preceeeds a shell command
!ls -1 ncas_data/

ls: cannot access 'ncas_data/': No such file or directory


So there are plenty of netCDF files to work with. Note there is a mixture of "classic" netCDF-3 and netCDF-4, as some futher shell commands illustrate:

In [3]:
# Note that in IPython ! preceeeds a shell command
!ncdump -k ncas_data/data1.nc
!ncdump -k ncas_data/data2.nc

ncdump: ncas_data/data1.nc: No such file or directory
ncdump: ncas_data/data2.nc: No such file or directory


Let's start, naturally, by importing the CF data tools modules. Note the standard alias used for `cfplot` e.g. within the module documentation:

In [4]:
import cfdm
import cf
import cfplot as cfp

Great. We are now all ready to go using these modules on the netCDF datasets!

----

## 1. From netCDF to field constructs and back

### Read in netCDF files, create a new field construct by modification of data and metadata and then write out the new field to a new netCDF file

In [7]:
# Read a data file
field_list = cf.read('../../ncas_data/ua.nc')

TypeError: Report._add_message() got an unexpected keyword argument 'variable'

In [ ]:
field = field_list[0]

In [ ]:
field

In [ ]:
print(field)  # more detail

In [ ]:
field.dump()   # maximal (metadata) detail!

In [ ]:
squared_field = field * field

In [ ]:
print(field.data)
print(squared_field.data)

In [ ]:
print(field.units)
print(squared_field.units)

In [ ]:
print(field.standard_name)
print(squared_field.standard_name)  # this will fail! (explanation to follow!)

In [ ]:
squared_field.standard_name = 'square_of_eastward_wind'

In [ ]:
print(field.standard_name)
print(squared_field.standard_name)  # this now does not fail, as we have re-assigned a standard name

We can write out field constructs into netCDF files in any combination we wish. Let's squared field to a netCDF file:

In [ ]:
cf.write(squared_field, 'squared_e_wind.nc')

In [ ]:
# Note that in IPython ! preceeeds a shell command
!ls

In [ ]:
! ncdump -h squared_e_wind.nc

*Here we have read in a field construct from netCDF, created a new field based on the other field's data and metadata, modified the metadata of the new field, and then written in out to a netCDF file.*

----

## 2. Basic data analysis, with plotting of results

### Plot the data before and after applying statistical collapses

In [ ]:
a = cf.read('ncas_data/qbo.nc')[0]

In [ ]:
print(a)

In [ ]:
b = a.collapse('maximum', axes='T')  # temporal maximum

In [ ]:
print(b)

In [ ]:
b_sub = b.subspace(X=30)
print(b)

In [ ]:
cfp.con(b_sub)

In [ ]:
cfp.con(b.subspace(X=0))

In [ ]:
c = a.collapse('mean', axes='X')  # horizontal mean

In [ ]:
print(c)

In [ ]:
c_sub = c.subspace(T=cf.dt('1979-01-16 09:00:00'))

cfp.con(c_sub)

*That was a demo of some very basic statistical collapsing and sub-spacing.**

----

## 3. Regridding domains, with plotting of results

### Plot the data before and after regridding across spherical and cartesian coordinate systems

#### a) Regridding across spherical coordinate systems: conservative method as an example

Read in two fields, ``f`` and ``g``, where ``f`` is gridded at about twice the resolution of ``g``:

In [ ]:
# Read in a precipitation field and inspect it
f = cf.read('ncas_data/precip_2010.nc')[0]
print(f)

In [ ]:
# Read in another, lower-resolution, precipitation field and inspect it
g = cf.read('ncas_data/model_precip_DJF_means_low_res.nc')[0]
print(g)

Regrid the first field to the grid of the second. We use the `regrids` method of cf-python.

In [ ]:
h_1 = f.regrids(g, method='patch')
h_2 = f.regrids(g, method='conservative')
h_1.equals(h_2)

Now let's inspect what we have, by plotting the field "before and after" (though actually we keep two different fields) the regridding:

In [ ]:
# Take some subspaces first:
f_sub = f[0]
h_1_sub = h_1[0]
h_2_sub = h_2[0]


# Customising the plots to look nicer
cfp.mapset()
#cfp.mapset(proj='robin')
cfp.cscale('rh_19lev')

cfp.gopen(rows=1, columns=2)
cfp.gpos(1)
cfp.con(f_sub, blockfill=True, lines=False, colorbar_orientation='vertical',
        title='Precipitation field before regridding')
cfp.gpos(2)
cfp.con(h_1_sub, blockfill=True, lines=False, colorbar_orientation='vertical',
        title='...and after regridding with patch recovery')
cfp.gclose()

print("Comparing results fom different regridding methods:")
cfp.con(h_2_sub - h_1_sub, lines=False)

As we expect, the regridded field resembles the original in its nature, but is at lower-resolution due to its new grid.

#### b) Regridding across cartesian coordinate systems: time series as an example

The term 'regridding' brings to mind a multi-dimensional grid e.g. over the earth's surface, but a 'grid' is really just a set of points in a multi-dimensional space. In 1D, this is just a series of data points.

Cartesian regridding can be used for 1 to 3 dimensions, so we can use it to "regrid" such a series, and let's use a time series as an example.

Again, start by reading in some (different) precipitation fields, in this case ``i`` and ``j`` which form a pair of time series with different domains/grids i.e. numbers of time data points:

In [ ]:
# Read in a precipitation field and inspect it
i = cf.read('ncas_data/precip_1D_yearly.nc')[0]
print(i)

In [ ]:
j = cf.read('ncas_data/precip_1D_monthly.nc')[0]
print(j)

Regrid linearly along the time axis 'T' and summarise the resulting field. This time, because we are working with cartesian coordinates, we need to use the `regridc` method on the field acting as the source domain.

For diversity, we use a different regridding method. Let's use linear interpolation, by setting `method='linear'`:

In [ ]:
k = i.regridc(j, axes='T', method='linear')
print(k)

Plot the time series before and after regridding

In [ ]:
cfp.gopen(rows=1, columns=2)
cfp.gpos(1)
cfp.lineplot(i, marker='o', color='red',
             title='Original time series... before regridding')
cfp.gpos(2)
cfp.lineplot(k, marker='o', color='blue', title='... and after regridding')
cfp.gclose()

In this case, we've seen that regridding can apply not just to multi-dimensional coordinates but to *data series* (which are *1D "grids"*).

As you can see, again the nature of the regridding output is preserved, but the granularity has changed, in this case becoming higher.

----

## 3. Manipulating hierarchical groups

### Create & inspect group structure for a netCDF-4 file, then flatten it out

*That was a quick demonstration of regridding using both the `regrids` and `regridc` methods for spherical and cartesian coordinate systems respectively, showcasing three different interpolation methods.*

We want to look at groups so let's read in some fields from a netCDF-4 dataset:

In [ ]:
# Note that in IPython ! preceeeds a shell command
!ncdump -k ncas_data/data1.nc

Take the first field from the `FieldList`:

In [ ]:
f = cf.read('ncas_data/data1.nc')[0]
print(f)

Let's see if there is any group structure already:

In [ ]:
f.nc_variable_groups()

So, we see that there is not. But, if we wanted groups, we could create some. A group structure that may be applied when writing to disk can be created from scratch with the netCDF interface, and cf-python provides methods that use this.

Here as an example we create a group structure, with `forecast` and `model` as named groups, and write it to disk:

In [ ]:
f.set_property('comment', 'some general comment')
f.nc_set_group_attribute('comment', 'I am part of the model group, a sub-group of forecast')
f.nc_set_variable_groups(['forecast', 'model'])

f.construct('time').nc_set_variable_groups(['forecast'])

cf.write(f, 'grouped.nc')

Let's just check that we wrote out our file by reading it back in again and checking (alternatively, verify with `ncdump`):

In [ ]:
g = cf.read('grouped.nc')[0]
print(g)
print(f)  # for comparison with original
g.equals(f)

In [ ]:
g.nc_variable_groups()

Those are the precise groups we just created, so all is good. We can also verify the groups have been created correctly by inspecting with `ncdump`:

In [ ]:
# Note that in IPython ! preceeeds a shell command
!ncdump -h grouped.nc

In [ ]:
g.nc_group_attributes(values=True)

In [ ]:
g.construct('latitude').nc_get_variable()

By default field constructs are written out to a dataset with their groups struct (if any) intact. It is always possible, however, to create a “flat” dataset, i.e. one without any sub-groups, just by setting the `group` keyword argument to `False`, like so:

In [ ]:
cf.write(g, 'flat.nc', group=False)

In [ ]:
h = cf.read('flat.nc')[0]
h

In [ ]:
# Note that in IPython ! preceeeds a shell command
!ncdump -h flat.nc

In [ ]:
h.nc_variable_groups()

In [ ]:
h.nc_group_attributes(values=True)

In [ ]:
f = cf.read('ncas_data/data1.nc')[0]
h.equals(f, verbose='detail')

Compare the comment attributes attached to each field:

In [ ]:
h.comment

But:

In [ ]:
f.comment

That concludes a quick demo of inspecting and manipulating netCDF-4 hierarchical groups, using cf-python.

----